# Configuración

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
from funciones_analisis.construccion_tabla import *
os.getcwd()

'C:\\Users\\DarioAlejandroLatorr\\OneDrive\\claro_20_abril_2021\\ENVIO_DARIO\\SEGMENTACIONES_2020_TATIANA_LEAL\\SEGMENTACIONES_2020'

# Importe de datos y construcción de tabla general

In [3]:
df_seg_nod_pap_hijo, df_fechas_seg = agregar_fechaseg_nodohijo('Nodos padre-hijo 2020.csv',lag_1_mes=False)
df_estrato = get_estrato('NODOS_ESTRATO_USUARIOS_2020.csv')
df_regional = pd.read_csv('DATA_IN/NODOS_REGIONAL_2020.csv',sep=';')
df_regional.columns = ['Nodo','REGIONAL']

In [4]:
var_interes = ['BAJAS','VENTAS','CHURN_VOL_SIN_MIG','CRECIMIENTO_NETO','USUARIOS']
lista_var_rel =  ['BAJAS','VENTAS','CRECIMIENTO_NETO']
df_vars_com = construir_df_vars(var_interes, lista_var_rel, df_seg_nod_pap_hijo)

dic_meses = get_dic_meses()
df_vars_com['MES_no'] = [dic_meses[mes] for mes in df_vars_com['MES']]

In [5]:
df_vars_com = df_vars_com.merge(df_estrato, how='left', on='Nodo')
df_vars_com = df_vars_com.merge(df_regional, how='left', on='Nodo')
df_vars_com = df_vars_com.merge(df_fechas_seg, how='left', on=['Nodo','MES_no'] )
df_vars_com.fillna(value={'SEGMENTADO':False}, inplace=True)

In [6]:
seg_2020 = df_vars_com.groupby('Nodo').agg({'SEGMENTADO':lambda x: max(x)==1}).reset_index()
seg_2020.rename(columns={'SEGMENTADO':'SEGMENTADO_2020'}, inplace=True)
df_vars_com = df_vars_com.merge(seg_2020, how='left', on=['Nodo'] )

In [7]:
df_vars_com['SEGMENTADO_n'] = df_vars_com['SEGMENTADO']*1
dic_mes_fecha = {mes:f'01/0{dic_meses[mes]}/2020' if dic_meses[mes] <10  else f'01/{dic_meses[mes]}/2020' for mes in dic_meses.keys()}
df_vars_com['MES'] = [dic_mes_fecha[x] for j,x in enumerate(df_vars_com['MES'])]
df_vars_com[['USUARIOS_ESTRATO_pred','ESTRATO_pred']] = df_vars_com[['USUARIOS_ESTRATO_pred','ESTRATO_pred']].fillna(value=-1)
ruta = 'NODOS_VAR_SEGMENTACIONES_2020.csv'
df_vars_com.to_csv(ruta, sep=';', decimal=',', index=False)

# Algunas verificaciones

## Nodos Padres e hijos (Algunos hijos no están en la tabla)

### T2M padre T2MA hijo

In [8]:
df_ven = pd.read_csv('DATA_IN/NODOS_VENTAS_2020.csv', sep=';')
df_ven[df_ven['Nodo']=='T2MA']

,Nodo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic
15183,T2MA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,11.0


In [9]:
df_ven[df_ven['Nodo']=='T2M']

,Nodo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic
15181,T2M,48.0,90.0,63.0,42.0,92.0,31.0,42.0,72.0,35.0,26.0,8.0,25.0


In [10]:
df_vars_com[df_vars_com['Nodo']=='T2M'][['Nodo','MES_no','VENTAS']].set_index('MES_no').transpose()

MES_no,1,2,3,4,5,6,7,8,9,10,11,12
Nodo,T2M,T2M,T2M,T2M,T2M,T2M,T2M,T2M,T2M,T2M,T2M,T2M
VENTAS,48.0,90.0,63.0,42.0,92.0,31.0,42.0,72.0,35.0,26.0,16.0,36.0


### P3c padre P3c1 hijo (NO info del hijo)

In [11]:
df_ven[df_ven['Nodo']=='P3C1']

,Nodo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic


In [12]:
df_ven[df_ven['Nodo']=='P3C']

,Nodo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic
12847,P3C,0.0,4.0,3.0,10.0,6.0,3.0,4.0,4.0,10.0,2.0,3.0,2.0


In [13]:
df_vars_com[df_vars_com['Nodo']=='P3C'][['Nodo','MES_no','VENTAS']].set_index('MES_no').transpose()

MES_no,1,2,3,4,5,6,7,8,9,10,11,12
Nodo,P3C,P3C,P3C,P3C,P3C,P3C,P3C,P3C,P3C,P3C,P3C,P3C
VENTAS,0.0,4.0,3.0,10.0,6.0,3.0,4.0,4.0,10.0,2.0,3.0,2.0


### ETU padre ETU1 hijo 

In [14]:
df_ven[df_ven['Nodo']=='ETU1']

,Nodo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic
7694,ETU1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,0.0


In [15]:
df_ven[df_ven['Nodo']=='ETU']

,Nodo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic
7693,ETU,68.0,72.0,61.0,80.0,112.0,116.0,113.0,123.0,108.0,71.0,51.0,69.0


In [16]:
df_vars_com[df_vars_com['Nodo']=='ETU'][['Nodo','MES_no','VENTAS']].set_index('MES_no').transpose()

MES_no,1,2,3,4,5,6,7,8,9,10,11,12
Nodo,ETU,ETU,ETU,ETU,ETU,ETU,ETU,ETU,ETU,ETU,ETU,ETU
VENTAS,68.0,72.0,61.0,80.0,112.0,116.0,113.0,123.0,108.0,71.0,57.0,69.0


## Descripción de variables numéricas

In [17]:
df_vars_com.describe()

,BAJAS,VENTAS,CHURN_VOL_SIN_MIG,CRECIMIENTO_NETO,USUARIOS,BAJAS_CORR,CHURN_VOL_SIN_MIG_CORR,CRECIMIENTO_NETO_CORR,BAJAS_rel,VENTAS_rel,CRECIMIENTO_NETO_rel,BAJAS_CORR_rel,CRECIMIENTO_NETO_CORR_rel,USUARIOS_ESTRATO_pred,ESTRATO_pred,SEGMENTADO_n
count,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000,202032.000000
mean,-8.422428,12.218109,-0.016839,3.795681,439.880811,-8.560609,-0.018490,3.657500,-0.025309,0.049906,0.024597,-0.026913,0.022992,395.835947,2.917082,0.017888
std,12.819919,17.249533,0.088689,15.660821,436.618486,12.680455,0.075529,15.476707,0.101054,0.535484,0.569546,0.089551,0.544038,400.200370,1.258385,0.132546
min,-710.000000,0.000000,-5.000000,-710.000000,0.000000,-710.000000,-5.000000,-710.000000,-5.000000,0.000000,-5.000000,-5.000000,-5.000000,-1.000000,-1.000000,0.000000
25%,-13.000000,1.000000,-0.017293,-2.000000,88.000000,-13.000000,-0.017293,-2.000000,-0.025994,0.001745,-0.006135,-0.025994,-0.006135,80.000000,2.000000,0.000000
50%,-5.000000,6.000000,-0.007804,1.000000,300.000000,-5.000000,-0.007804,0.000000,-0.013986,0.018182,0.000000,-0.013986,0.000000,269.000000,3.000000,0.000000
75%,0.000000,17.000000,0.000000,7.000000,696.000000,0.000000,0.000000,7.000000,0.000000,0.036058,0.020619,0.000000,0.019608,604.000000,4.000000,0.000000
max,133.000000,903.000000,12.500000,914.000000,5984.000000,0.000000,0.000000,903.000000,12.500000,95.500000,108.000000,0.000000,95.500000,4118.000000,6.000000,1.000000


# Sampleador

Se implementa un método para encontrar una muestra de nodos no segmentados durante el 2020 lo más parecida posible a los nodos segmentados (buqueda de un grupo control). Para enocntrar la muestra "parecida" se resamplea buscando una distribución de número de usuarios similar en los segementados y no segmentados  en el mes de enero.

In [18]:
import seaborn as sns
import matplotlib.pyplot as plt 
from scipy.stats import ks_2samp

In [19]:
%matplotlib qt

***Descipción de número de usuarios en los nodos segmentados en enero***

In [20]:
mask = (df_vars_com['SEGMENTADO_2020']) & (df_vars_com['MES_no']==1) 
df_vars_com.loc[mask,['USUARIOS']].describe(np.arange(0.9,1,0.01)).transpose()

,count,mean,std,min,50%,90%,91%,92%,93%,94%,95%,96%,97%,98%,99%,max
USUARIOS,544.0,1162.875,598.740208,0.0,1108.0,1932.8,1957.64,2049.92,2071.0,2115.76,2205.45,2280.4,2351.43,2396.9,2618.28,3273.0


***Descipción de número de usuarios en los nodos NO segmentados en enero***

In [21]:
mask = (~df_vars_com['SEGMENTADO_2020']) & (df_vars_com['MES_no']==1) 
df_vars_com.loc[mask,['USUARIOS']].describe(np.arange(0.9,1,0.01)).transpose()

,count,mean,std,min,50%,90%,91%,92%,93%,94%,95%,96%,97%,98%,99%,max
USUARIOS,16292.0,391.286828,386.62607,0.0,265.0,956.0,986.81,1022.0,1058.0,1100.0,1146.0,1197.0,1275.27,1377.18,1522.18,4977.0


In [22]:
mask = (~df_vars_com['SEGMENTADO_2020']) & (df_vars_com['MES_no']==1) 
df_vars_com.loc[mask,['USUARIOS']].describe(np.arange(0.99,1,0.001)).transpose()

,count,mean,std,min,50%,99%,99.1%,99.2%,99.3%,99.4%,99.5%,99.6%,99.7%,99.8%,99.9%,100%,max
USUARIOS,16292.0,391.286828,386.62607,0.0,265.0,1522.18,1545.381,1566.688,1601.778,1628.0,1674.0,1710.836,1784.016,1870.836,1983.762,4977.0,4977.0


In [23]:
mes = 1
list_vars = ['USUARIOS','VENTAS','BAJAS_CORR']
for var_ in list_vars:
    plt.figure()
    sns.histplot(data=df_vars_com[df_vars_com['MES_no']==mes], x=var_,hue='SEGMENTADO_2020', common_norm=False, stat='probability',bins=50)
    plt.show()

***Se resamplea usando el número de usuarios en el mes de enero, pero se restringe el análisis a nodos con hasta 2000 usuarios***

In [24]:
mask_enero = (df_vars_com['MES_no']==mes)
mask_limit_usuarios = (df_vars_com['USUARIOS']<2000)
lista_columns = ['Nodo','USUARIOS','SEGMENTADO_2020'] 
df_vars_ene = df_vars_com.loc[(mask_enero & mask_limit_usuarios),lista_columns].copy()


df_var_ene_seg = df_vars_ene[df_vars_ene['SEGMENTADO_2020']].copy()
df_var_ene_NO_seg = df_vars_ene[~df_vars_ene['SEGMENTADO_2020']].copy()

# remuestreo con reemplazo (es necesario repetir para que en la cola se pesen diferente)
df_var_ene_NO_seg_sample = resample(df_var_ene_NO_seg, df_var_ene_seg, 'USUARIOS', maximum=2060, step=60, 
                                    show=False, sample_size=1_000, semilla=12345)

# semillas y resultados nivel de sinficancia 10% (se buscan muestras que pasaran el test de kolmogorov)

# 17 ---> pasa test kolmogorov-smirnov --- intervalo did : 0.001  0.004
# 345 ---> pasa test kolmogorov-smirnov --- intervalo did : 0.001 0.005
# 4328 ---> pasa test kolmogorov-smirnov --- intervalo did : 0.000 0.003
# 46991 ---> pasa test kolmogorov-smirnov --- intervalo did : 0.001 0.003
# 87849 ---> pasa test kolmogorov-smirnov --- intervalo did : 0.000 0.003
# 156789 ---> pasa test kolmogorov-smirnov --- intervalo did : 0.001  0.004
# 3456791 ---> pasa test kolmogorov-smirnov --- intervalo did : 0.000 0.003
# 43456791 ---> pasa test kolmogorov-smirnov --- intervalo did : 0.001 0.003
# 735641845 ---> pasa test kolmogorov-smirnov --- intervalo did : 8.71e-05 0.003
# 621242680 ---> pasa test kolmogorov-smirnov --- intervalo did : -0.001 0.002

# eliminando los repetidos
#df_var_ene_NO_seg_sample = df_var_ene_NO_seg_sample.drop_duplicates(subset='Nodo')
df_ene_analisis = pd.concat([df_var_ene_seg,df_var_ene_NO_seg_sample])

df_ene_analisis.rename(columns={'USUARIOS':'USUARIOS_ENERO'}, inplace=True)
df_ene_analisis.drop(columns='SEGMENTADO_2020', inplace=True)
df_ene_analisis['IDENTIFICADOR_SAMPLE'] = np.arange(len(df_ene_analisis))
df_analisis = df_ene_analisis.merge(df_vars_com, how='left', on='Nodo')

In [25]:
df_revisar_mes = df_analisis[df_analisis['MES_no']==mes]
for var_ in list_vars:
    plt.figure()
    sns.histplot(data=df_revisar_mes, x=var_,hue='SEGMENTADO_2020', common_norm=False, stat='probability',bins=30)
    plt.show()

In [26]:
print(f'{100*"+"}\nPRUEBAS KOLMOGOROV SMIRNOV COMPARACIÓN DE DISTRIBUCIONES (ANTES MUESTREO)\n{100*"+"}')
df_mes_completo = df_vars_com[df_vars_com['MES_no']==mes]
for var_ in list_vars:
    print(f'{40*"-"} {var_} {40*"-"}')
    data_seg = df_mes_completo.loc[(df_mes_completo['SEGMENTADO_2020']),var_]
    data_no_seg = df_mes_completo.loc[~(df_mes_completo['SEGMENTADO_2020']),var_]
    print(ks_2samp(data_seg,data_no_seg))
print(f'{100*"+"}\nPRUEBAS KOLMOGOROV SMIRNOV COMPARACIÓN DE DISTRIBUCIONES (DESPUÉS MUESTREO)\n{100*"+"}')
for var_ in list_vars:
    print(f'{40*"-"} {var_} {40*"-"}')
    data_seg = df_revisar_mes.loc[(df_revisar_mes['SEGMENTADO_2020']),var_]
    data_no_seg = df_revisar_mes.loc[~(df_revisar_mes['SEGMENTADO_2020']),var_]
    print(ks_2samp(data_seg,data_no_seg))

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
PRUEBAS KOLMOGOROV SMIRNOV COMPARACIÓN DE DISTRIBUCIONES (ANTES MUESTREO)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
---------------------------------------- USUARIOS ----------------------------------------
KstestResult(statistic=0.5647241157695584, pvalue=7.533383336538081e-159)
---------------------------------------- VENTAS ----------------------------------------
KstestResult(statistic=0.4557149124073887, pvalue=2.368549480916393e-100)
---------------------------------------- BAJAS_CORR ----------------------------------------
KstestResult(statistic=0.43108490634161845, pvalue=2.503074105811246e-89)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
PRUEBAS KOLMOGOROV SMIRNOV COMPARACIÓN DE DISTRIBUCIONES (DESPUÉS MUESTREO)
++++++++++++++++++++++++++++++++++++++++++++++++++++

In [27]:
for var_ in list_vars:
    plt.figure()
    sns.boxplot(data=df_mes_completo, x=var_, y='ESTRATO_pred', hue='SEGMENTADO_2020', orient='h')
    plt.show()

for var_ in list_vars:
    plt.figure()
    sns.boxplot(data=df_revisar_mes, x=var_, y='ESTRATO_pred', hue='SEGMENTADO_2020', orient='h')
    plt.show()

In [28]:
df_analisis['CRECIMIENTO NETO POR CADA 1000 USUARIOS'] = df_analisis['CRECIMIENTO_NETO_CORR_rel']*1000
ruta = 'NODOS_SAMPLEADO_ANALISIS_SEGMENTACIONES_2020.csv'
df_analisis.to_csv(ruta, sep=';', decimal=',', index=False)

# REGRESIÓN LINEAL (DIFERENCIAS en DIFERENCIAS)

Se realiza un análisis de regresión con las variables trimestre 1 o 4, segmentado 2020 True o False y la interacción entre las dos para cuantificar el impacto de la segmentación y determinar si es significativo. 

Se asume que las tendencias en nodos segmentados y no segmentados eran similares, en cierto grado el sampleo realizado justifica ese supuesto.


In [29]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [30]:
df_reg_dif_crec_neto_corr_rel = get_data_mod_lin_trimestre(df_analisis, 'CRECIMIENTO_NETO_CORR_rel') 

In [31]:
nivel_significancia = 0.1
X = df_reg_dif_crec_neto_corr_rel[['TRIMESTRE','DID']] # ,'SEGMENTADO_2020' quitando la sobreparametrización
y = df_reg_dif_crec_neto_corr_rel['CRECIMIENTO_NETO_CORR_rel']
X = sm.add_constant(X)
results = sm.OLS(y,X).fit()
results.summary(alpha=nivel_significancia)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     CRECIMIENTO_NETO_CORR_rel   R-squared:                       0.010
Model:                                   OLS   Adj. R-squared:                  0.009
Method:                        Least Squares   F-statistic:                     15.12
Date:                       Fri, 16 Jul 2021   Prob (F-statistic):           2.93e-07
Time:                               08:46:50   Log-Likelihood:                 6048.8
No. Observations:                       2992   AIC:                        -1.209e+04
Df Residuals:                           2989   BIC:                        -1.207e+04
Df Model:                                  2                                         
Covariance Type:                   nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|       [0.05       0.95]
------------------------------------------------------------------------------
const          0.0062      0.001      7.488      0.000       0.005       0.008
TRIMESTRE     -0.0072      0.001     -5.493      0.000      -0.009      -0.005
DID            0.0039      0.002      2.222      0.026       0.001       0.007
==============================================================================
Omnibus:                     2591.330   Durbin-Watson:                   1.842
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         23494580.965
Skew:                           2.422   Prob(JB):                         0.00
Kurtosis:                     437.092   Cond. No.                         3.81
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Simulaciones intervalos de confianza para el efecto

In [32]:
np.random.seed(1)
var_analisis = 'CRECIMIENTO_NETO_CORR_rel'
no_simulaciones = 50  
arreglo_semillas = np.random.randint(low=1,high=2**30,size=no_simulaciones)
nivel_significancia = 0.1
list_df = []
for j,semilla in enumerate(arreglo_semillas):
    if (j+1)%10==0:
        print(f'{j} de {no_simulaciones}')
    # sampleando
    df_var_ene_NO_seg_sample = resample(df_var_ene_NO_seg, df_var_ene_seg, 'USUARIOS', maximum=2060, step=60, 
                                        show=False, sample_size=1_000, semilla=semilla)
    # uniendo nodos segmentados con nodos no segmentados que escogieron en el sampleo
    df_ene_analisis = pd.concat([df_var_ene_seg,df_var_ene_NO_seg_sample])
    df_ene_analisis.rename(columns={'USUARIOS':'USUARIOS_ENERO'}, inplace=True)
    df_ene_analisis.drop(columns='SEGMENTADO_2020', inplace=True)
    df_ene_analisis['IDENTIFICADOR_SAMPLE'] = np.arange(len(df_ene_analisis))
    # agregando toda la info de los nodos segmentados y no segmentados para el análisis
    df_analisis = df_ene_analisis.merge(df_vars_com, how='left', on='Nodo')
    # alistando datos para la regresión
    df_reg_var = get_data_mod_lin_trimestre(df_analisis, var_analisis)
    # ajustando modelo
    X = df_reg_var[['TRIMESTRE','SEGMENTADO_2020','DID']]
    y = df_reg_var[var_analisis]
    X = sm.add_constant(X)
    results = sm.OLS(y,X).fit()
    results.summary(alpha=nivel_significancia)
    # encontrando y agregando el intervalo de confianza para DID
    list_df.append(results.conf_int().loc[['DID']].rename(index={'DID': semilla}))

df_simulaciones_int_confianza = pd.concat(list_df)
df_simulaciones_int_confianza.columns=['LIMITE INFERIOR','LIMITE SUPERIOR']
m = (df_simulaciones_int_confianza['LIMITE INFERIOR']+df_simulaciones_int_confianza['LIMITE SUPERIOR'])/2
df_simulaciones_int_confianza['ESTIMACION MEDIA'] = m
ruta = f'SIMULACIONES_INTCONDID_{var_analisis}_SEG_2020.csv'
df_simulaciones_int_confianza = df_simulaciones_int_confianza*1000
df_simulaciones_int_confianza.sort_values('ESTIMACION MEDIA', inplace=True)
df_simulaciones_int_confianza['INDEX'] = np.arange(1,no_simulaciones+1)
df_simulaciones_int_confianza.to_csv(ruta, sep=';', decimal=',', index=False)

9 de 50
19 de 50
29 de 50
39 de 50
49 de 50


In [33]:
df_simulaciones_int_confianza

,LIMITE INFERIOR,LIMITE SUPERIOR,ESTIMACION MEDIA,INDEX
795511699,-1.193670,6.315077,2.560704,1
413652245,-1.401866,7.519665,3.058900,2
147806607,0.093841,7.044494,3.569167,3
729053693,-0.362895,7.686507,3.661806,4
1013994433,-0.208478,7.535223,3.663373,5
629559426,-0.369879,8.018624,3.824372,6
77964602,-0.544207,8.470026,3.962909,7
398576446,-0.586414,8.602732,4.008159,8
796884250,0.315019,7.707641,4.011330,9
271757670,-0.073677,8.256283,4.091303,10
